In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/commontest/dataset/val/no/422.npy
/kaggle/input/commontest/dataset/val/no/331.npy
/kaggle/input/commontest/dataset/val/no/2434.npy
/kaggle/input/commontest/dataset/val/no/280.npy
/kaggle/input/commontest/dataset/val/no/970.npy
/kaggle/input/commontest/dataset/val/no/2467.npy
/kaggle/input/commontest/dataset/val/no/2424.npy
/kaggle/input/commontest/dataset/val/no/745.npy
/kaggle/input/commontest/dataset/val/no/100.npy
/kaggle/input/commontest/dataset/val/no/1200.npy
/kaggle/input/commontest/dataset/val/no/1170.npy
/kaggle/input/commontest/dataset/val/no/1876.npy
/kaggle/input/commontest/dataset/val/no/719.npy
/kaggle/input/commontest/dataset/val/no/1041.npy
/kaggle/input/commontest/dataset/val/no/193.npy
/kaggle/input/commontest/dataset/val/no/1191.npy
/kaggle/input/commontest/dataset/val/no/461.npy
/kaggle/input/commontest/dataset/val/no/2196.npy
/kaggle/input/commontest/dataset/val/no/63.npy
/kaggle/input/commontest/dataset/val/no/1364.npy
/kaggle/input/commontest/datase

In [53]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, loguniform
from sklearn.metrics import (
    accuracy_score, 
    roc_auc_score, 
    roc_curve, 
    confusion_matrix,
    classification_report
)

In [54]:
class EfficientHyperparameterTuner:
    """Efficient Hyperparameter Tuning Framework"""
    def __init__(self, config_class, data_dir):
        self.config_class = config_class
        self.data_dir = data_dir
        
        # Define randomized search space
        self.param_distributions = {
            'lr': loguniform(1e-5, 1e-3),  # Log-uniform distribution for learning rate
            'batch_size': [32, 64, 128],
            'weight_decay': loguniform(1e-4, 1e-2),
            'model_name': ['resnet18', 'efficientnet_b0'],
            'dropout_rate': uniform(0.2, 0.5)
        }
    
    def tune(self, max_trials=10, epochs=5):
        """Efficient hyperparameter tuning"""
        best_score = 0
        best_config = None
        results = []
        
        # Prepare base dataset once
        train_df, val_df = self._prepare_base_data()
        
        for trial in range(max_trials):
            # Randomly sample hyperparameters
            params = self._sample_params()
            print(f"\n--- Trial {trial + 1} ---")
            print("Hyperparameters:", params)
            
            # Create configuration
            config = self._create_config(params, epochs)
            
            # Train and evaluate
            try:
                val_auc, trial_details = self._run_trial(config, train_df, val_df)
                
                # Store and track results
                result = {
                    'hyperparameters': params,
                    'val_auc': val_auc,
                    'details': trial_details
                }
                results.append(result)
                
                # Update best configuration
                if val_auc > best_score:
                    best_score = val_auc
                    best_config = params
                
                print(f"Validation AUC: {val_auc:.4f}")
            
            except Exception as e:
                print(f"Trial failed: {e}")
                continue
        
        # Visualize and save results
        self._analyze_results(results)
        
        return best_config, best_score
    
    def _prepare_base_data(self):
        """Prepare base dataset"""
        # Reuse data preparation logic from ModelTrainer
        trainer = ModelTrainer(self.config_class())
        return trainer.prepare_data(self.data_dir)
    
    def _sample_params(self):
        """Randomly sample hyperparameters"""
        return {
            'lr': self.param_distributions['lr'].rvs(),
            'batch_size': np.random.choice(self.param_distributions['batch_size']),
            'weight_decay': self.param_distributions['weight_decay'].rvs(),
            'model_name': np.random.choice(self.param_distributions['model_name']),
            'dropout_rate': self.param_distributions['dropout_rate'].rvs()
        }
    
    def _create_config(self, params, epochs):
        """Create configuration with sampled parameters"""
        config = self.config_class()
        config.lr = params['lr']
        config.batch_size = int(params['batch_size'])
        config.weight_decay = params['weight_decay']
        config.model_name = params['model_name']
        config.dropout_rate = params['dropout_rate']
        config.epochs = epochs
        return config
    
    def _run_trial(self, config, train_df, val_df):
        """Run a single trial with given configuration"""
        # Initialize trainer
        trainer = ModelTrainer(config)
        
        # Create data loaders
        train_loader, val_loader = trainer.create_dataloaders(train_df, val_df)
        
        # Train the model
        model, predictions, labels, history = trainer.train(train_loader, val_loader)
        
        # Calculate performance metric
        val_auc = history['val_auc'][-1]
        
        return val_auc, {
            'train_loss': history['train_loss'],
            'val_loss': history['val_loss'],
            'learning_rates': history['learning_rates']
        }
    
    def _analyze_results(self, results):
        """Analyze and visualize tuning results"""
        # Convert results to DataFrame
        df = pd.DataFrame([
            {**r['hyperparameters'], 'val_auc': r['val_auc']} 
            for r in results
        ])
        
        # Create visualization
        plt.figure(figsize=(15, 10))
        
        # Learning Rate vs Performance
        plt.subplot(2, 2, 1)
        sns.scatterplot(data=df, x='lr', y='val_auc', hue='model_name')
        plt.title('Learning Rate vs Performance')
        plt.xscale('log')
        
        # Batch Size vs Performance
        plt.subplot(2, 2, 2)
        sns.boxplot(data=df, x='batch_size', y='val_auc')
        plt.title('Batch Size vs Performance')
        
        # Weight Decay vs Performance
        plt.subplot(2, 2, 3)
        sns.scatterplot(data=df, x='weight_decay', y='val_auc', hue='model_name')
        plt.title('Weight Decay vs Performance')
        plt.xscale('log')
        
        # Model Name vs Performance
        plt.subplot(2, 2, 4)
        sns.boxplot(data=df, x='model_name', y='val_auc')
        plt.title('Model Architecture vs Performance')
        plt.xticks(rotation=45)
        
        plt.tight_layout()
        plt.savefig('hyperparameter_tuning_results.png')
        plt.close()

In [61]:
class ModelTrainer:
    """Optimized Model Trainer with Reduced Epochs"""
    def __init__(self, config):
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def prepare_data(self, data_dir):
        """Prepare training and validation datasets"""
        def _get_image_paths(base_path, classes):
            paths = []
            labels = []
            for label, cls in enumerate(classes):
                class_path = os.path.join(base_path, cls)
                for img_file in os.listdir(class_path):
                    paths.append(os.path.join(class_path, img_file))
                    labels.append(label)
            return paths, labels

        # Directory 
        train_path = os.path.join(data_dir, 'train')
        classes = ['no', 'sphere', 'vort']
        
        paths, labels = _get_image_paths(train_path, classes)
        
        df = pd.DataFrame({
            'data_path': paths,
            'target': labels
        })
        
        # Split into train and validation
        train_df, val_df = train_test_split(
            df, 
            test_size=0.1, # 90 : 10 train-test split specified as per document 
            stratify=df['target'], 
            random_state=self.config.seed
        )
        
        return train_df, val_df

    def create_dataloaders(self, train_df, val_df):
        """Create PyTorch DataLoaders"""
        train_dataset = LensDataset(train_df)
        val_dataset = LensDataset(val_df)
        
        train_loader = DataLoader(
            train_dataset, 
            batch_size=self.config.batch_size, 
            shuffle=True, 
            num_workers=self.config.num_workers
        )
        
        val_loader = DataLoader(
            val_dataset, 
            batch_size=self.config.batch_size, 
            shuffle=False, 
            num_workers=self.config.num_workers
        )
        
        return train_loader, val_loader
    
    
    def train(self, train_loader, val_loader):
        """Streamlined training loop with early stopping"""
        # Initialize model with configurable dropout
        model = LensClassificationModel(self.config).to(self.device)
        
        # Loss and Optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(
            model.parameters(), 
            lr=self.config.lr, 
            weight_decay=self.config.weight_decay
        )
        
        # Learning Rate Scheduler
        scheduler = ReduceLROnPlateau(
            optimizer, 
            mode='max',
            factor=0.5,
            patience=2,
            verbose=True
        )
        
        # Early Stopping
        patience = 3
        min_delta = 0.001
        best_val_auc = 0
        counter = 0
        
        # Training Tracking
        training_history = {
            'train_loss': [],
            'val_loss': [],
            'val_auc': [],
            'learning_rates': []
        }
        
        for epoch in range(self.config.epochs):
            # Training Phase
            model.train()
            train_losses = []
            
            for images, labels in train_loader:
                images, labels = images.to(self.device), labels.to(self.device)
                
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
                train_losses.append(loss.item())
            
            # Validation Phase
            model.eval()
            val_losses = []
            all_preds = []
            all_labels = []
            
            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(self.device), labels.to(self.device)
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    
                    val_losses.append(loss.item())
                    preds = torch.softmax(outputs, dim=1)
                    all_preds.append(preds.cpu().numpy())
                    all_labels.append(labels.cpu().numpy())
            
            # Aggregate predictions and labels
            all_preds = np.concatenate(all_preds)
            all_labels = np.concatenate(all_labels)
            
            # Calculate metrics
            val_auc = roc_auc_score(all_labels, all_preds, multi_class='ovr')
            
            # Log metrics
            avg_train_loss = np.mean(train_losses)
            avg_val_loss = np.mean(val_losses)
            
            training_history['train_loss'].append(avg_train_loss)
            training_history['val_loss'].append(avg_val_loss)
            training_history['val_auc'].append(val_auc)
            
            # Get current learning rate
            current_lr = optimizer.param_groups[0]['lr']
            training_history['learning_rates'].append(current_lr)
            
            # Learning Rate Scheduling
            scheduler.step(val_auc)
            
            # Early Stopping
            if val_auc > best_val_auc + min_delta:
                best_val_auc = val_auc
                counter = 0
                # Save best model
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'best_val_auc': best_val_auc
                }, 'best_model.pth')
            else:
                counter += 1
            
            # Stop if no improvement
            if counter >= patience:
                print(f"Early stopping triggered after {epoch+1} epochs")
                break
            
            # Print epoch summary
            print(f"Epoch {epoch+1}/{self.config.epochs}")
            print(f"Train Loss: {avg_train_loss:.4f}")
            print(f"Val Loss: {avg_val_loss:.4f}")
            print(f"Val AUC: {val_auc:.4f}")
            print(f"Current LR: {current_lr:.6f}")
        
        return model, all_preds, all_labels, training_history

In [70]:
class ModelVisualizer:
    """Visualization utilities for model performance"""
    @staticmethod
    def plot_training_history(history, save_path='training_history.png'):
        """Plot training and validation loss/AUC"""
        plt.figure(figsize=(12, 4))
        
        # Loss Plot
        plt.subplot(1, 2, 1)
        plt.plot(history['train_loss'], label='Train Loss')
        plt.plot(history['val_loss'], label='Validation Loss')
        plt.title('Training and Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        
        # AUC Plot
        plt.subplot(1, 2, 2)
        plt.plot(history['val_auc'], label='Validation AUC', color='green')
        plt.title('Validation AUC')
        plt.xlabel('Epoch')
        plt.ylabel('AUC Score')
        plt.legend()
        
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()

    @staticmethod
    def plot_confusion_matrix(labels, predictions, class_names, save_path='confusion_matrix.png'):
        """Create and save confusion matrix visualization"""
        # Get predicted class labels
        pred_labels = np.argmax(predictions, axis=1)
        
        # Compute confusion matrix
        cm = confusion_matrix(labels, pred_labels)
        
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=class_names, 
                    yticklabels=class_names)
        plt.title('Confusion Matrix')
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()

    @staticmethod
    def plot_roc_curve(labels, predictions, class_names, save_path='roc_curve.png'):
        """Plot ROC curves for multiclass classification"""
        plt.figure(figsize=(10, 8))
        
        # One-vs-Rest ROC Curves
        for i in range(len(class_names)):
            # Create binary labels for current class
            binary_labels = (labels == i).astype(int)
            class_preds = predictions[:, i]
            
            # Compute ROC curve
            fpr, tpr, _ = roc_curve(binary_labels, class_preds)
            
            # Calculate AUC
            roc_auc = roc_auc_score(binary_labels, class_preds)
            
            # Plot ROC curve
            plt.plot(fpr, tpr, 
                     label=f'ROC curve (class: {class_names[i]}, AUC = {roc_auc:.2f})')
        
        plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc="lower right")
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()



In [79]:
def main():
    # Configuration
    config_class = DeepLensConfig 
    data_dir = '/kaggle/input/commontest/dataset'
    
    # Defining classes explicitly
    CLASS_NAMES = ['no', 'sphere', 'vort']

    # Efficient Hyperparameter Tuning
    tuner = EfficientHyperparameterTuner(config_class, data_dir)
    best_config, best_score = tuner.tune(
        max_trials=10,  
        epochs=5        
    )

    print("\nBest Hyperparameters:")
    print(best_config)
    print(f"Best Validation AUC: {best_score:.4f}")


    # Manually Hardcoded afer Hyperparameter Tuning 
    BEST_PARAMS = {
        'lr':  0.0008936514993794424, # Learning rate
        'batch_size': 32,  # Batch size
        'weight_decay': 0.0001812043338125292, # Weight decay
        'model_name': 'efficientnet_b0', # Model architecture
        'dropout_rate': 0.4516472894261153   # Dropout rate
    }

    # Create configuration with manual parameters
    config = config_class()
    config.lr = BEST_PARAMS['lr']
    config.batch_size = BEST_PARAMS['batch_size']
    config.weight_decay = BEST_PARAMS['weight_decay']
    config.model_name = BEST_PARAMS['model_name']
    config.dropout_rate = BEST_PARAMS['dropout_rate']

    # Prepare data
    trainer = ModelTrainer(config)
    train_df, val_df = trainer.prepare_data(data_dir)
    
    # Create data loaders
    train_loader, val_loader = trainer.create_dataloaders(train_df, val_df)
    
    # Train model with best configuration
    model, predictions, labels, history = trainer.train(train_loader, val_loader)
    
    # Visualization
    visualizer = ModelVisualizer()
    
    # 1. Training History Plot
    visualizer.plot_training_history(history)
    
    # 2. Confusion Matrix (pass class names explicitly)
    visualizer.plot_confusion_matrix(labels, predictions, CLASS_NAMES)
    
    # 3. ROC Curves (pass class names explicitly)
    visualizer.plot_roc_curve(labels, predictions, CLASS_NAMES)
    
    # 4. Save Best Model with Additional Metadata
    model_save_path = 'best_model_full.pth'
    torch.save({
        'model_state_dict': model.state_dict(),
        'best_config': BEST_PARAMS,
        'class_names': CLASS_NAMES,
        'training_history': history
    }, model_save_path)
    
    # 5. Generate Detailed Performance Report
    print("\nDetailed Classification Report:")
    pred_labels = np.argmax(predictions, axis=1)
    print(classification_report(labels, pred_labels, target_names=CLASS_NAMES))

if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/10
Train Loss: 0.9088
Val Loss: 0.7603
Val AUC: 0.8725
Current LR: 0.000894
Epoch 2/10
Train Loss: 0.5301
Val Loss: 0.4182
Val AUC: 0.9541
Current LR: 0.000894
Epoch 3/10
Train Loss: 0.3739
Val Loss: 0.3804
Val AUC: 0.9643
Current LR: 0.000894
Epoch 4/10
Train Loss: 0.3036
Val Loss: 0.2956
Val AUC: 0.9746
Current LR: 0.000894
Epoch 5/10
Train Loss: 0.2528
Val Loss: 0.2525
Val AUC: 0.9809
Current LR: 0.000894
Epoch 6/10
Train Loss: 0.2190
Val Loss: 0.2620
Val AUC: 0.9799
Current LR: 0.000894
Epoch 7/10
Train Loss: 0.1933
Val Loss: 0.2824
Val AUC: 0.9786
Current LR: 0.000894
Epoch 8/10
Train Loss: 0.1845
Val Loss: 0.3115
Val AUC: 0.9828
Current LR: 0.000894
Epoch 9/10
Train Loss: 0.1533
Val Loss: 0.3046
Val AUC: 0.9803
Current LR: 0.000894
Epoch 10/10
Train Loss: 0.1388
Val Loss: 0.3205
Val AUC: 0.9826
Current LR: 0.000894

Detailed Classification Report:
              precision    recall  f1-score   support

          no       0.86      0.99      0.92      1000
      sphere     